In [2]:
%reset -f
def get_api_key():
    import os
    from dotenv import load_dotenv
    # Unset OPENAI_API_KEY if it exists
    os.environ.pop('OPENAI_API_KEY', None)
    # Provide path and ignore currently set environment variable
    dotenv_path = '.env'
    load_dotenv(dotenv_path=dotenv_path, override=True)
    # Access the environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    if api_key is None:
        print("API_KEY not found.")
    return api_key


In [3]:
# Agents - LLM as Reasoning Engine
import openai
import langchain
import langchain_community
import docarray
import tiktoken
import sys

print(f'python version = {sys.version}')
print(f'langchain version = {langchain.__version__}')
print(f'openai version = {openai.__version__}')
print(f'docarray version = {docarray.__version__}')
print(f'LangChain Community Version = {langchain_community.__version__}')

# *** Course dependencies ***
# python version = 3.11.6
# langchain version = 0.1.5
# openai version = 1.10.0
# docarray version = 0.32.1
# LangChain Community Version = 0.0.17
# *** Course dependencies ***

python version = 3.11.6 (main, Feb  2 2024, 13:53:38) [GCC 11.4.0]
langchain version = 0.1.5
openai version = 1.10.0
docarray version = 0.32.1
LangChain Community Version = 0.0.17


In [4]:
# !pip install -U wikipedia

In [45]:
from langchain.agents import tool
from datetime import date

In [5]:
# https://github.com/langchain-ai/langchain/issues/15370
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

In [10]:
import openai
openai.api_key = get_api_key()
from langchain_openai import ChatOpenAI
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0, model=llm_model)

In [52]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [49]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

# Construct the ReAct agent
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "what is 25% of 1000? Only give the final number."})

# agent = initialize_agent(
#     tools, 
#     llm, 
#     agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     handle_parsing_errors=True,
#     verbose = True)



> Entering new AgentExecutor chain...
I can use the calculator to find the answer to this math problem.
Action: Calculator
Action Input: 25% of 1000Answer: 250.0I now know the final answer
Final Answer: 250.0

> Finished chain.


{'input': 'what is 25% of 1000? Only give the final number.',
 'output': '250.0'}

In [60]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [58]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [61]:
langchain.debug=True
agent.invoke({"input":f"Sort these customers by \
last name and then first name \
and print the output: {customer_list}"})
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

In [93]:
from langchain.tools import BaseTool, StructuredTool, tool
from datetime import date
@tool
def dateTool(text: str) -> str:
    """Returns todays date, use this for any\
    questions related to knowing todays date or time.\
    The input can be anything, this function will always return todays\
    date"""
    return str(date.today())
print(dateTool("date"))

2024-02-08


In [95]:
tools += [dateTool]
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input":"What is the date today?"})



> Entering new AgentExecutor chain...
I can use the dateTool to find out the date today.
Action: dateTool
Action Input: "What is the date today?"2024-02-08I now know the final answer
Final Answer: The date today is 2024-02-08.

> Finished chain.


{'input': 'What is the date today?', 'output': 'The date today is 2024-02-08.'}